In [ ]:
#attempt to improve the NN
#add the local and the global view construct coz transit false positive mismatch seems to be a major problem
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from keras import backend as K

In [ ]:
X_train=np.loadtxt('training_data/Xtrain_seg_mask.csv',delimiter=',')
Y_train=np.loadtxt('training_data/Ytrain_seg_mask.csv',delimiter=',')

Y_train=Y_train.reshape(len(Y_train),4800,2)


In [ ]:
X_train=np.asarray([(row-np.median(row))/(-row[np.argmin(row)]+np.median(row)) for row in X_train])


In [ ]:
Xtrain, Xtest, Ytrain, Ytest= train_test_split(X_train, Y_train, test_size=0.2)
print(Xtrain.shape,Ytrain.shape,Xtest.shape,Ytest.shape)



In [ ]:

vararr=np.random.randint(0,len(Xtrain),size=10)
fig,ax=plt.subplots(10,2,figsize=(10,20))
for i in range(0,10):
    ax[i][0].plot(Xtrain[vararr[i]])
    ax[i][1].plot(Ytrain[vararr[i],:,0])
    ax[i][1].plot(Ytrain[vararr[i],:,1])
plt.show


In [ ]:


def dice_coeff(y_true, y_pred):
    smooth = 1.
    sh = tf.shape(y_true)
    y_true_int = tf.reshape(y_true, [sh[0], sh[1]*sh[2]])
    y_pred_int = tf.reshape(y_pred, [sh[0], sh[1]*sh[2]])
    w = 1 - tf.reduce_sum(y_true,(1,2)) / 9600.
    w = tf.reshape(w, [len(w),1])
    # Flatten
    y_true_f = tf.reshape(tf.multiply(y_true_int,w), [-1])
    y_pred_f = tf.reshape(tf.multiply(y_pred_int,w), [-1])
    #y_pred_int_f = tf.reshape(tf.matmul(w, y_pred_int), [-1])
    #y_true_f = tf.reshape(y_true, [-1])
    y_pred_f_2 = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f_2)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) +  tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = keras.losses.binary_crossentropy(y_true, y_pred) + generalized_dice_coeff(y_true, y_pred)
    return loss



def generalized_dice_coeff(y_true, y_pred):
    # Compute weights: "the contribution of each label is corrected by the inverse of its volume"
    w = tf.reduce_sum(Ytest,(0,1))
    w= w/tf.linalg.norm(w)
    w = 1 / (w  + 0.00001)
    w=tf.cast(w,tf.float32)
    numerator = y_true * y_pred
    numerator = w * K.sum(numerator, (0, 1))
    numerator = K.sum(numerator)

    denominator = y_true + y_pred
    denominator = w * K.sum(denominator, (0, 1))
    denominator = K.sum(denominator)

    gen_dice_coef = numerator / denominator

    return 1 - 2 * gen_dice_coef

In [ ]:

IMG_SIZE=6000
#add hidden layers
conv_ip = keras.layers.Input(shape=(IMG_SIZE,),name='Input')
xi=keras.layers.Reshape((IMG_SIZE, 1), input_shape=(IMG_SIZE,),name='reshape_1')(conv_ip)
#xi=keras.layers.Cropping1D(cropping=(100, 100))(xi)
xi=keras.layers.BatchNormalization()(xi)

x1=keras.layers.Conv1D(16,kernel_size=5,strides=3,activation='tanh',padding='same',name='second_conv16_5')(xi)
x1=keras.layers.Conv1D(16,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv16_5')(x1)
x1=keras.layers.BatchNormalization()(x1)
c1=keras.layers.MaxPool1D(3,strides=2,data_format='channels_last',padding='same',name='maxpool_1')(x1)   #size 1000

x2=keras.layers.Conv1D(32,kernel_size=3,strides=1,activation='tanh',padding='same',name='second_conv32_5')(c1)
x2=keras.layers.Conv1D(32,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv32_5')(x2)
x2=keras.layers.BatchNormalization()(x2)
c2=keras.layers.MaxPool1D(3,strides=2,data_format='channels_last',padding='same',name='maxpool_2')(x2)    #size 500

ao = keras.layers.Conv1DTranspose(filters=32, kernel_size=3, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_1")(c2)
ao = keras.layers.Conv1DTranspose(filters=32, kernel_size=3, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_2")(ao)
ao=keras.layers.Flatten()(ao)
ao=keras.layers.Dense(200,activation='tanh',name='adding_this')(ao)

x3=keras.layers.Conv1D(64,kernel_size=3,strides=1,activation='tanh',padding='same',name='second_conv64_5')(c2)
x3=keras.layers.Conv1D(64,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv64_5')(x3)
x3=keras.layers.BatchNormalization()(x3)
x3=keras.layers.MaxPool1D(3,strides=2,data_format='channels_last',padding='same',name='maxpool_3')(x3)    #size 250

x3=keras.layers.Conv1D(128,kernel_size=3,strides=1,activation='tanh',padding='same',name='second_conv128_5')(x3)
x3=keras.layers.Conv1D(128,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv128_5')(x3)
x3=keras.layers.BatchNormalization()(x3)
x3=keras.layers.MaxPool1D(3,strides=2,data_format='channels_last',padding='same',name='maxpool_4')(x3)    #size 125


x3 = keras.layers.Conv1DTranspose(filters=64, kernel_size=3, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_1")(x3)
x3 = keras.layers.Conv1DTranspose(filters=64, kernel_size=3, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_2")(x3)

x3 = keras.layers.Conv1DTranspose(filters=32, kernel_size=3, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_3")(x3)
x3 = keras.layers.Conv1DTranspose(filters=32, kernel_size=3, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_4")(x3)
tx2 = keras.layers.Concatenate(axis=1)([c2,x3]) #size=400
tx2 = keras.layers.Conv1D(32,kernel_size=3,strides=2,activation='tanh',padding='same',name='maxpoolt_2')(tx2)
tx2=keras.layers.BatchNormalization()(tx2)

#tx2 = keras.layers.Conv1DTranspose(filters=16, kernel_size=5, activation='tanh', padding="same", strides=1, name="decoder_conv_tran_5")(tx2)
#tx2 = keras.layers.Conv1DTranspose(filters=16, kernel_size=5  , activation='tanh', padding="same", strides=1, name="decoder_conv_tran_6")(tx2)

tx2=keras.layers.Flatten()(x3)
tx2=keras.layers.Dense(128,activation='relu')(tx2)
tx2=keras.layers.Dense(128,activation='relu')(tx2)
tx2=keras.layers.Dense(1,activation='relu')(tx2)
conv_op=keras.layers.Dense(2,activation='softmax',name='missing_this')(tx2)



#convNN = keras.Model(inputs=[conv_ip,conv_ipl], outputs=conv_op,name='Convolutional_NN')
convNN = keras.Model(inputs=conv_ip, outputs=[ao,conv_op],name='Convolutional_NN')


convNN.summary()
convNN.compile(optimizer=keras.optimizers.Adam(learning_rate=0.000005), 
    loss={'missing_this': 'categorical_crossentropy','adding_this':keras.losses.Huber(delta=0.1)},metrics={'missing_this':'accuracy'})
print(' ')

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history=convNN.fit(np.array(Xtrain),[np.array(Itrain),np.array(Ytrain)], batch_size=64, epochs=70 , verbose=1 ,
    shuffle=True,validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')  
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.savefig('present_segment.png')
plt.show()

In [ ]:

from sklearn.metrics import mean_squared_error
#convNN.load_weights('long_hard_battle_eq.h5')
pred_op=convNN.predict(np.array(Xtest))

fig,ax=plt.subplots(6,2,figsize=(15,15))
plt.style.use('seaborn-bright')
plt.suptitle('Network Output')
ar=np.random.randint(len(Xtest),size=10)
#ar=[0,1,2,3,8,7]
ax[0][0].set_title('Generated')
ax[0][1].set_title('Original')
for i in range(0,6):
    ax[i][0].plot(Xtest[ar[i]],color='gray',ls='None',marker='.',label='data')
    ax[i][0].plot(-pred_op[ar[i],:,0],color='black',ls='None',marker='.',label='fps')
    ax[i][0].plot(-pred_op[ar[i],:,1],color='green',ls='None',marker='.',label='pl')
    
    
    ax[i][1].plot(Xtest[ar[i]],color='gray',ls='None',marker='.',label='data')
    ax[i][1].plot(-Ytest[ar[i],:,0],color='black',ls='None',marker='.',label='fps')
    ax[i][1].plot(-Ytest[ar[i],:,1],color='green',ls='None',marker='.',label='pl')
    
    #ax[i][1].plot(pred_op_mod[ar[i]],color='black',ls='None',marker='.')
    ax[i][0].legend('flux')
    ax[i][0].set_ylim(-1.05,0.05)
    ax[i][1].set_ylim(-1.05,0.05)
    ax[i][0].legend()
    ax[i][1].legend()
ax[5][0].set_xlabel('Phase')
ax[5][1].set_xlabel('Phase')

#plt.savefig('present_itsamust')


In [ ]:
class_Ytest=[]
class_Ypred=[]
for el in pred_op:
    pl=np.array(el[:,0]>0.20).sum()
    fps=np.array(el[:,1]>0.35).sum()
    if(pl>fps):class_Ypred.append(1)
    else: class_Ypred.append(0)

for el in Ytest:
    pl=np.array(el[:,0]>0.20).sum()
    fps=np.array(el[:,1]>0.35).sum()
    if(pl>fps):class_Ytest.append(1)
    else: class_Ytest.append(0)
    #4800,2
#[print(class_Ypred[i],class_Ytest[i]) for i in range(0,len(class_Ytest))]

cm=confusion_matrix(class_Ytest,class_Ypred)
print(cm)

In [ ]:
#different approach to CM
class_Ytest=[]
class_Ypred=[]
for el in pred_op:
    pl=np.array(el[:,0]>0.20).sum()
    fps=np.array(el[:,1]>0.30).sum()
    if(pl>3*fps):class_Ypred.append([1,0])
    elif(fps>3*pl):class_Ypred.append([0,1])
    else: class_Ypred.append([1,1])

for el in Ytest:
    pl=np.array(el[:,0]>0.20).sum()
    fps=np.array(el[:,1]>0.30).sum()
    if(pl*fps):class_Ytest.append([1,1])
    elif(pl):class_Ytest.append([1,0])
    else: class_Ytest.append([0,1])
    #4800,2
#[print(class_Ypred[i],class_Ytest[i]) for i in range(0,len(class_Ytest))]


cm=confusion_matrix(np.asarray(class_Ytest)[:,0],np.asarray(class_Ypred)[:,0])
print(cm)

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc

fpr , tpr, thes= roc_curve(class_Ytest,class_Ypred,pos_label=1)
print(auc(fpr,tpr))
print(fpr.shape,tpr.shape,thes.shape)
plt.style.use('seaborn-bright')
plt.figure(figsize=(7,5))
plt.plot(fpr,tpr)
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

In [ ]:
mask_train=convNN.predict(np.array(Xtrain))
mask_test=convNN.predict(np.array(Xtest))
mask_train=np.asarray([[1 if max(el[0],el[1])>0.3 else 0 for el in x ] for x in mask_train])
mask_test=np.asarray([[1 if max(el[0],el[1])>0.3 else 0 for el in x ] for x in mask_test])
Itrain=Xtrain*mask_train
Itest=Xtest*mask_test
cltrain=[]
cltest=[]

for el in Ytrain:
    pl=np.array(el[:,0]>0.5).sum()
    fps=np.array(el[:,1]>0.5).sum()
    if(pl>fps):cltrain.append([1,0])
    else: cltrain.append([0,1])

for el in Ytest:
    pl=np.array(el[:,0]>0.5).sum()
    fps=np.array(el[:,1]>0.5).sum()
    if(pl>fps):cltest.append([1,0])
    else: cltest.append([0,1])

print(np.asarray(Itrain).shape,np.asarray(cltrain).shape)

In [ ]:
mask_test=convNN.predict(np.array(Xtest))
mask_test=np.asarray([[1 if max(el[0],el[1])>0.5 else 0 for el in x ] for x in mask_test])
mask_test_true=np.asarray([[1 if max(el[0],el[1])>0.5 else 0 for el in x ] for x in Ytest])

In [ ]:
def smoothbin(arr):
    for m in range(5,len(arr)-5):
        if(arr[m]==0):
            if(arr[m-1]==-1 and arr[m+1]==-1): 
                arr[m]=-1
            elif(arr[m-2]==-1 and arr[m+2]==-1): 
                arr[m]=-1
            elif(arr[m-3]==-1 and arr[m+3]==-1): 
                arr[m]=-1
            elif(arr[m-4]==-1 and arr[m+4]==-1): 
                arr[m]=-1
            elif(arr[m-5]==-1 and arr[m+5]==-1): 
                arr[m]=-1
    return arr

fig,ax=plt.subplots(4,1,figsize=(10,10))
plt.style.use('seaborn-bright')
plt.suptitle('sample reconstructions')
#ar=np.random.randint(len(Xtest),size=5)
ar=[0,7,2,3]

for i in range(0,4):
    ax[i].plot(Xtest[ar[i]],color='gray',ls='None',marker='.',label='raw')
    ax[i].plot(mask_test_true[ar[i]]*min(Xtest[ar[i]]),color='blue',ls='-',label='generated')
    ax[i].plot(smoothbin(mask_test[ar[i]])*min(Xtest[ar[i]])*0.8,color='black',label='predict')
    ax[i].legend()
    ax[i].set_ylabel('flux')
    #ax[i][1].plot(pred_op_mod[ar[i]],color='black',ls='None',marker='.')
    
    #ax[i][0].legend()
ax[3].set_xlabel('Time')
plt.savefig('present_rec.png')

In [ ]:
convNN.save_weights('long_hard_battle_eq.h5')
convNN.save('Model_long_hard_battle_eq.h5')

In [ ]:

vararr=np.random.randint(0,len(Xtrain),size=10)
fig,ax=plt.subplots(10,1,figsize=(10,20))
for i in range(0,10):
    ax[i].plot( Itrain[vararr[i]][2000:3000],label=Ytrain[vararr[i]])
plt.show

In [ ]:
IM_SIZE=4800
#add hidden layers
class_ip = keras.layers.Input(shape=(IM_SIZE,),name='Input')

m=keras.layers.Reshape((IM_SIZE, 1), input_shape=(IM_SIZE,),name='reshape_1')(class_ip)
m=keras.layers.BatchNormalization()(m)
m=keras.layers.Conv1D(16,kernel_size=3,strides=2,activation='tanh',padding='same',name='second_conv16_5')(m)
m=keras.layers.Conv1D(16,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv16_5')(m)

m=keras.layers.MaxPool1D(3,strides=2 ,data_format='channels_last',padding='same',name='maxpool_1')(m)
m=keras.layers.Conv1D(32,kernel_size=3,strides=1,activation='tanh',padding='same',name='second_conv32_5')(m)
m=keras.layers.Conv1D(32,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv32_5')(m)

m=keras.layers.MaxPool1D(3,strides=2 ,data_format='channels_last',padding='same',name='maxpool_2')(m)
m=keras.layers.Conv1D(64,kernel_size=3,strides=1,activation='tanh',padding='same',name='second_conv64_5')(m)
m=keras.layers.Conv1D(64,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv64_5')(m)

m=keras.layers.MaxPool1D(3,strides=2 ,data_format='channels_last',padding='same',name='maxpool_4')(m)
m=keras.layers.Conv1D(128,kernel_size=3,strides=1,activation='tanh',padding='same',name='second_conv128_5')(m)
m=keras.layers.Conv1D(128,kernel_size=3,strides=1,activation='tanh',padding='same',name='third_conv128_5')(m)

m=keras.layers.MaxPool1D(3,strides=2 ,data_format='channels_last',padding='same',name='maxpool_3')(m)
m=keras.layers.Flatten(name='flat_1')(m)
m=keras.layers.Dense(256,name='dense_layer_3',activation='relu')(m)
m=keras.layers.Dense(256,name='dense_layer_5',activation='relu')(m)
m=keras.layers.Dense(256,name='dense_layer_7',activation='relu')(m)
m=keras.layers.Dense(1,name='dense_layer_4',activation='relu')(m)
class_op=keras.layers.Dense(2,name='dense_layer_6',activation='softmax')(m)


#convNN = keras.Model(inputs=[conv_ip,conv_ipl], outputs=conv_op,name='Convolutional_NN')
classify = keras.Model(inputs=class_ip, outputs=class_op,name='Classifier_NN')


classify.summary()
classify.compile(optimizer=keras.optimizers.Adam(learning_rate=0.000005), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#history=convNN.fit([np.array(Xtrain),np.array(Xtrainl)],np.array(Ytrain), batch_size=64, epochs=40, verbose=VERBOSE, validation_split=0.12,callbacks=[es_callback])
history2=classify.fit(np.asarray(Itrain),np.asarray(cltrain), batch_size=64, epochs=40 , verbose=1 , shuffle=True,validation_split=0.15)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy\n Can the NN work with the raw chunk?')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#  plt.savefig('rec_manual_mask.png')
plt.show()

In [ ]:
test_loss, test_acc = classify.evaluate(np.array(Itest), np.array(cltest))
print('Test accuracy:', test_acc)
Ypred_raw=classify.predict(np.array(Itest))
Ypred=np.argmax(Ypred_raw, axis=1)
Ytest_new=np.argmax(cltest,axis=1)
cm = confusion_matrix(Ytest_new, Ypred)
print(cm)